**Проект 1. Кто хочет стать миллионером кинопроката?**  
    *выполнил: Закиров Наиль (DST-53)*  
    *2021-03-07*  


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
504,tt0213149,140000000,449220945,Pearl Harbor,Ben Affleck|Josh Hartnett|Kate Beckinsale|Cuba...,Michael Bay,"December 7, 1941 - A day that shall live in in...",The lifelong friendship between Rafe McCawley ...,183,History|Romance|War,Jerry Bruckheimer Films|Touchstone Pictures,5/21/2001,6.6,2001
1187,tt0770828,225000000,662845518,Man of Steel,Henry Cavill|Amy Adams|Michael Shannon|Diane L...,Zack Snyder,You will believe that a man can fly.,A young boy learns that he has extraordinary p...,143,Action|Adventure|Fantasy|Science Fiction,Legendary Pictures|Warner Bros.|DC Entertainme...,6/12/2013,6.5,2013
169,tt2937898,20000000,12007070,A Most Violent Year,Oscar Isaac|Jessica Chastain|Albert Brooks|Ely...,J.C. Chandor,The result is never in question. Just the path...,A thriller set in New York City during the win...,125,Action|Thriller|Crime|Drama,Participant Media|Imagenation Abu Dhabi FZ|Fil...,12/30/2014,6.4,2014
1792,tt0455760,20000000,22217407,Dead Silence,Ryan Kwanten|Donnie Wahlberg|Bob Gunton|Amber ...,James Wan,You scream. You die.,Jamie returns to his hometown in search of ans...,89,Horror,Universal Pictures|Twisted Pictures|Evolution ...,3/16/2007,6.2,2007
1048,tt1433811,10000000,3428048,Disconnect,Jason Bateman|Hope Davis|Frank Grillo|Paula Pa...,Henry Alex Rubin,Look up.,Disconnect interweaves multiple storylines abo...,115,Drama|Thriller,Lionsgate|Liddell Entertainment|LD Entertainme...,9/11/2012,6.7,2012


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
imdb_id                 1889 non-null object
budget                  1889 non-null int64
revenue                 1889 non-null int64
original_title          1889 non-null object
cast                    1889 non-null object
director                1889 non-null object
tagline                 1889 non-null object
overview                1889 non-null object
runtime                 1889 non-null int64
genres                  1889 non-null object
production_companies    1889 non-null object
release_date            1889 non-null object
vote_average            1889 non-null float64
release_year            1889 non-null int64
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [5]:
answers = {} # создадим словарь для ответов

In [6]:
# Функция выбирающая номер ответа в списке вариантов ответов по полученному ответу. Автоматизируем поиск вариантов ответа.
def option_search(answer_options, my_answer):
    """
    (lst, lst|str) -> str
    Choose number of answer in options by answer.
    """
    answer = ""
    i = 1 
    j = 0
    for option in answer_options:
        true_answer = True
        for item in option:
            if item in my_answer:
                true_answer = true_answer and True
            else:
                true_answer = true_answer and False
        if true_answer:           
            j += 1
            if j == 1:
                answer += f"{i}. {', '.join(option)}"
            else:
                answer += f"|{i}. {', '.join(option)}"
        i += 1
    return answer
      


In [7]:
# Функция сохранения результата в словаре результатов.
def answer_to(answers, task_number, answer_str):
    """
    (dict, num, str) 
    Save str to dict with key num.
    """
    answers[task_number] = answer_str
    return



In [8]:
# Конвертация числового представления месяца в текстовый.
def month_convert(month_number):
    """
    (str) -> str
    Convert number of month to month name.
    """
    months = {'1': 'Январь', 
              '2': 'Февоаль', 
              '3': 'Март', 
              '4': 'Апрель', 
              '5': 'Май',
              '6': 'Июнь',
              '7': 'Июль',
              '8': 'Август',
              '9': 'Сентябрь',
              '10': 'Октябрь',
              '11': 'Ноябрь',
              '12': 'Декабрь'
             }
    return months[month_number]



In [9]:
# Создание пар элементов из исходного списка (сочетание без повторений по 2)
def pair_up(source_list, pair_list):
    """
    (lst, changed_target_lst) 
    Math combination from list by pair.
    """
    sorted_source = sorted(source_list)
    i = 1
    for item in sorted_source[:-1]:
        for pair in sorted_source[i:]:
            pair_list.append(f"{item} & {pair}")
        i += 1
    return



In [10]:
task_number = 0

In [11]:
 # Получим множество уникальных жанров.
genres_unique = set(data.genres.str.split('|').sum())
genres_unique

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'Thriller',
 'War',
 'Western'}

In [12]:
# Добавим столбец с прибылью в исходный датасет.
data = data.assign(profit = data.revenue - data.budget)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 15 columns):
imdb_id                 1889 non-null object
budget                  1889 non-null int64
revenue                 1889 non-null int64
original_title          1889 non-null object
cast                    1889 non-null object
director                1889 non-null object
tagline                 1889 non-null object
overview                1889 non-null object
runtime                 1889 non-null int64
genres                  1889 non-null object
production_companies    1889 non-null object
release_date            1889 non-null object
vote_average            1889 non-null float64
release_year            1889 non-null int64
profit                  1889 non-null int64
dtypes: float64(1), int64(5), object(9)
memory usage: 221.5+ KB


# 1. У какого фильма из списка самый большой бюджет?

In [14]:
task_number = 1
answer_options_1 = [["The Dark Knight Rises (tt1345836)"],
                ["Spider-Man 3 (tt0413300)"],
                ["Avengers: Age of Ultron (tt2395427)"],
                ["The Warrior's Way (tt1032751)"],
                ["Pirates of the Caribbean: On Stranger Tides (tt1298650)"]]

In [15]:
# Решение:
my_answer_1 = data.original_title[data.budget == data.budget.max()].iloc[0] + \
    " (" + (data.imdb_id[data.budget == data.budget.max()].iloc[0]) + ")"
my_answer_1

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [16]:
answer_to(answers, str(task_number), option_search(answer_options_1, my_answer_1))
answers[str(task_number)]

'5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [17]:

idx = data.budget.sort_values().tail(1).index[0]

In [18]:
f"{data.original_title[idx]} ({data.imdb_id[idx]})" 

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [19]:
task_number = 2
answer_options_2 = [['The Lord of the Rings: The Return of the King (tt0167260)'],
                    ['Gods and Generals (tt0279111)'],
                    ['King Kong (tt0360717)'],
                    ['Pearl Harbor (tt0213149)'],
                    ['Alexander (tt0346491)']]

In [20]:
my_answer_2 = data.original_title[data.runtime == data.runtime.max()].iloc[0] + \
    " (" + (data.imdb_id[data.runtime == data.runtime.max()].iloc[0]) + ")"

In [21]:
my_answer_2

'Gods and Generals (tt0279111)'

In [22]:
answer_to(answers, str(task_number), option_search(answer_options_2, my_answer_2))
answers[str(task_number)]

'2. Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [23]:
task_number = 3
answer_options_3 = [["Home on the Range (tt0299172)"],
                    ["The Jungle Book 2 (tt0283426)"],
                    ["Winnie the Pooh (tt1449283)"],
                    ["Corpse Bride (tt0121164)"],
                    ["Hoodwinked! (tt0443536)"]]

In [24]:
my_answer_3 = data.original_title[data.runtime == data.runtime.min()].iloc[0] + \
    " (" + (data.imdb_id[data.runtime == data.runtime.min()].iloc[0]) + ")"

In [25]:
my_answer_3

'Winnie the Pooh (tt1449283)'

In [26]:
answer_to(answers, str(task_number), option_search(answer_options_3, my_answer_3))

In [27]:
answers[str(task_number)]

'3. Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [28]:
task_number = 4
answer_options_4 = [['115'],
                    ['110'],
                    ['105'],
                    ['120'],
                    ['100']]

In [29]:
my_answer_4 = round(data.runtime.mean()) 

In [30]:
my_answer_4

110

In [31]:
answer_to(answers, str(task_number), option_search(answer_options_4, str(my_answer_4)))

In [32]:
answers[str(task_number)]

'2. 110'

# 5. Каково медианное значение длительности фильмов? 

In [33]:
task_number = 5
answer_options_5 = [['107'],
                    ['112'],
                    ['101'],
                    ['120'],
                    ['115']]

In [34]:
my_answer_5 = round(data.runtime.median())

In [35]:
my_answer_5

107

In [36]:
answer_to(answers, str(task_number), option_search(answer_options_5, str(my_answer_5)))

In [37]:
answers[str(task_number)]

'1. 107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [38]:
task_number = 6
answer_options_6 = [['The Avengers (tt0848228)'],
                    ['Minions (tt2293640)'],
                    ['Star Wars: The Force Awakens (tt2488496)'],
                    ['Furious 7 (tt2820852)'],
                    ['Avatar (tt0499549)']]

In [39]:
# лучше код получения столбца profit вынести в Предобработку что в начале
my_answer_6 = data.original_title[data.profit == data.profit.max()].iloc[0] + \
    " (" + (data.imdb_id[data.profit == data.profit.max()].iloc[0]) + ")"

In [40]:
my_answer_6

'Avatar (tt0499549)'

In [41]:
answer_to(answers, str(task_number), option_search(answer_options_6, my_answer_6))

answers[str(task_number)]

'5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [42]:
task_number = 7
answer_options_7 = [['Supernova (tt0134983)'],
                    ['Frozen (tt2294629)'],
                    ['Flushed Away (tt0424095)'],
                    ['The Adventures of Pluto Nash (tt0180052)'],
                    ['The Lone Ranger (tt1210819)']]

In [43]:
my_answer_7 = data.original_title[data.profit == data.profit.min()].iloc[0] + \
    " (" + (data.imdb_id[data.profit == data.profit.min()].iloc[0]) + ")"

In [44]:
my_answer_7


'The Lone Ranger (tt1210819)'

In [45]:
answer_to(answers, str(task_number), option_search(answer_options_7, my_answer_7))

answers[str(task_number)]

'5. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [46]:
task_number = 8
answer_options_8 = [['1478'],
                    ['1520'],
                    ['1241'],
                    ['1135'],
                    ['1398']]

In [47]:
my_answer_8 = len(data[data.revenue > data.budget])

In [48]:
my_answer_8

1478

In [49]:
answer_to(answers, str(task_number), option_search(answer_options_8, str(my_answer_8)))

answers[str(task_number)]

'1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [50]:
task_number = 9
answer_options_9 = [['Madagascar: Escape 2 Africa (tt0479952)'],
                    ['Iron Man (tt0371746)'],
                    ['Kung Fu Panda (tt0441773)'],
                    ['The Dark Knight (tt0468569)'],
                    ['Mamma Mia! (tt0795421)']]

In [51]:
data_2008 = data[data.release_year == 2008]
my_answer_9 = data_2008.original_title[data_2008.revenue == data_2008.revenue.max()].iloc[0] + \
    " (" + data_2008.imdb_id[data_2008.revenue == data_2008.revenue.max()].iloc[0] + ")"

In [52]:
my_answer_9

'The Dark Knight (tt0468569)'

In [53]:
answer_to(answers, str(task_number), option_search(answer_options_9, my_answer_9))

answers[str(task_number)]

'4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [54]:
task_number = 10
answer_options_10 = [["Winter's Tale (tt1837709)"],
                    ["Stolen (tt1656186)"],
                    ["Broken City (tt1235522)"],
                    ["Upside Down (tt1374992)"],
                    ["The Lone Ranger (tt1210819)"]]

In [55]:
data_2012_2014 = data[(data.release_year >= 2012) & (data.release_year <= 2014)]
my_answer_10 = data_2012_2014.original_title[data_2012_2014.profit == data_2012_2014.profit.min()].iloc[0] + \
    " (" + (data_2012_2014.imdb_id[data_2012_2014.profit == data_2012_2014.profit.min()].iloc[0]) + ")"

In [56]:
my_answer_10

'The Lone Ranger (tt1210819)'

In [57]:
answer_to(answers, str(task_number), option_search(answer_options_10, my_answer_10))

answers[str(task_number)]

'5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [58]:
task_number = 11
answer_options_11 = [["Action"],
                    ["Adventure"],
                    ["Drama"],
                    ["Comedy"],
                    ["Thriller"]]

In [59]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [60]:
my_answer_11 = pd.Series(data.genres.str.split('|').sum()).value_counts().index[0]

In [61]:
my_answer_11

'Drama'

In [62]:
answer_to(answers, str(task_number), option_search(answer_options_11, my_answer_11))

answers[str(task_number)]

'3. Drama'

ВАРИАНТ 2


In [63]:
max(genres_unique, key=lambda x:data.genres.str.split('|').sum().count(x))


'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [64]:
task_number = 12
answer_options_12 = [["Drama"],
                    ["Comedy"],
                    ["Action"],
                    ["Thriller"],
                    ["Adventure"]]

In [65]:
s_genres_profit = data.genres[data.profit > 0]
s_genres_profit

0       Action|Adventure|Science Fiction|Thriller
1       Action|Adventure|Science Fiction|Thriller
2              Adventure|Science Fiction|Thriller
3        Action|Adventure|Science Fiction|Fantasy
4                           Action|Crime|Thriller
                          ...                    
1871                                       Comedy
1875                        Fantasy|Comedy|Family
1877                  Action|Comedy|Drama|Romance
1883                         Romance|Comedy|Drama
1884             Adventure|Action|Science Fiction
Name: genres, Length: 1478, dtype: object

In [66]:
my_answer_12 = pd.Series(s_genres_profit.str.split('|').sum()).value_counts().head(1).index[0]
my_answer_12

'Drama'

In [67]:
answer_to(answers, str(task_number), option_search(answer_options_12, my_answer_12))

answers[str(task_number)]

'1. Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [68]:
task_number = 13
answer_options_13 = [["Steven Spielberg"],
                    ["Christopher Nolan"],
                    ["David Yates"],
                    ["James Cameron"],
                    ["Peter Jackson"]]

In [69]:
df_director_revenue = data[['director', 'revenue']]
df_director_revenue

,director,revenue
0,Colin Trevorrow,1513528810
1,George Miller,378436354
2,Robert Schwentke,295238201
3,J.J. Abrams,2068178225
4,James Wan,1506249360
...,...,...
1884,Bryan Singer,157299717
1885,Uli Edel,13555988
1886,Des McAnuff,35134820
1887,Diane Keaton,36037909


In [70]:
df_director_revenue_separated = pd.DataFrame({'director':[], 'revenue':[]})
df_director_revenue_separated

,director,revenue


In [71]:
k = 0
for i, j in df_director_revenue.iterrows():
    for director in j[0].split('|'):
        df_director_revenue_separated.loc[k] = [director, j[1]]
        k += 1 

In [72]:
df_director_revenue_separated.head()

,director,revenue
0,Colin Trevorrow,1.513529e+09
1,George Miller,3.784364e+08
2,Robert Schwentke,2.952382e+08
3,J.J. Abrams,2.068178e+09
4,James Wan,1.506249e+09


In [73]:
df_director_revenue_separated.groupby('director').revenue.sum().sort_values().tail(1)

director
Peter Jackson    6.490594e+09
Name: revenue, dtype: float64

In [74]:
my_answer_13 = df_director_revenue_separated.groupby('director').revenue.sum().sort_values().tail(1).index[0]
my_answer_13

'Peter Jackson'

In [75]:
answer_to(answers, str(task_number), option_search(answer_options_13, my_answer_13))

answers[str(task_number)]

'5. Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [76]:
task_number = 14
answer_options_14 = [["Ridley Scott"],
                    ["Guy Ritchie"],
                    ["Robert Rodriguez"],
                    ["Quentin Tarantino"],
                    ["Tony Scott"]]

In [77]:
data_action = data[data.genres.str.contains("Action")]
pd.Series(data_action.director.str.split('|').sum()).value_counts().head(1)

Robert Rodriguez    9
dtype: int64

In [78]:
my_answer_14 = pd.Series(data_action.director.str.split('|').sum()).value_counts().head(1).index[0]
my_answer_14

'Robert Rodriguez'

In [79]:
answer_to(answers, str(task_number), option_search(answer_options_14, my_answer_14))

answers[str(task_number)]

'3. Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [80]:
task_number = 15
answer_options_15 = [["Nicolas Cage"],
                    ["Tom Hardy"],
                    ["Chris Hemsworth"],
                    ["Jim Sturgess"],
                    ["Emma Stone"]]

In [81]:
data_2012 = data[data.release_year == 2012]
df_2012_actor_revenue = data_2012[['cast', 'revenue']]
df_2012_actor_revenue

,cast,revenue
970,Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chr...,1519557910
971,Kate Beckinsale|Stephen Rea|Michael Ealy|Theo ...,132400000
972,Christian Bale|Michael Caine|Gary Oldman|Anne ...,1081041287
973,Jamie Foxx|Christoph Waltz|Leonardo DiCaprio|K...,425368238
974,Daniel Craig|Judi Dench|Javier Bardem|Ralph Fi...,1108561013
...,...,...
1076,Ashley Greene|Sebastian Stan|Tom Felton|Julian...,9627492
1077,Robert Pattinson|Sarah Gadon|Juliette Binoche|...,6063556
1078,Milla Jovovich|Sienna Guillory|Michelle Rodrig...,240159255
1079,Whitney Houston|Cee Lo Green|Derek Luke|Mike E...,24637469


In [82]:
df_2012_actor_revenue_separated = pd.DataFrame({'actor':[], 'revenue':[]})
df_2012_actor_revenue_separated

,actor,revenue


In [83]:
k = 0
for i, j in df_2012_actor_revenue.iterrows():
    for actor in j[0].split('|'):
        df_2012_actor_revenue_separated.loc[k] = [actor, j[1]]
        k += 1 

In [84]:
df_2012_actor_revenue_separated.head()

,actor,revenue
0,Robert Downey Jr.,1.519558e+09
1,Chris Evans,1.519558e+09
2,Mark Ruffalo,1.519558e+09
3,Chris Hemsworth,1.519558e+09
4,Scarlett Johansson,1.519558e+09


In [85]:
df_2012_actor_revenue_separated.groupby('actor').revenue.sum().sort_values().tail(1)

actor
Chris Hemsworth    2.027451e+09
Name: revenue, dtype: float64

In [86]:
my_answer_15 = df_2012_actor_revenue_separated.groupby('actor').revenue.sum().sort_values().tail(1).index[0]
my_answer_15

'Chris Hemsworth'

In [87]:
answer_to(answers, str(task_number), option_search(answer_options_15, my_answer_15))

answers[str(task_number)]

'3. Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [88]:
task_number = 16
answer_options_16 = [["Tom Cruise"],
                    ["Mark Wahlberg"],
                    ["Matt Damon"],
                    ["Angelina Jolie"],
                    ["Adam Sandler"]]

In [89]:
my_answer_16 = pd.Series(data[data.budget > data.budget.mean()].cast.str.split('|').sum()).value_counts().index[0]
my_answer_16

'Matt Damon'

In [90]:
answer_to(answers, str(task_number), option_search(answer_options_16, my_answer_16))

answers[str(task_number)]

'3. Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [91]:
task_number = 17
answer_options_17 = [["Drama"],
                    ["Action"],
                    ["Thriller"],
                    ["Adventure"],
                    ["Crime"]]

In [92]:
pd.Series(data[data.cast.str.contains("Nicolas Cage", na=False)].genres.str.split('|').sum()).value_counts()

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Mystery             3
Family              3
Animation           3
History             2
Horror              1
War                 1
Romance             1
dtype: int64

In [93]:
my_answer_17 = pd.Series(data[data.cast.str.contains("Nicolas Cage", na=False)].genres.str.split('|').sum()).\
                value_counts().head(1).index[0]
my_answer_17

'Action'

In [94]:
answer_to(answers, str(task_number), option_search(answer_options_17, my_answer_17))

answers[str(task_number)]

'2. Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [95]:
task_number = 18
answer_options_18 = [["K-19: The Widowmaker (tt0267626)"],
                    ["Next (tt0435705)"],
                    ["Twisted (tt0315297)"],
                    ["The Love Guru (tt0811138)"],
                    ["The Fighter (tt0964517)"]]

In [96]:
paramount_pict = data[data.production_companies.str.contains("Paramount Pictures")]
paramount_pict[paramount_pict.profit == paramount_pict.profit.min()].original_title

925    K-19: The Widowmaker
Name: original_title, dtype: object

In [97]:
my_answer_18 = paramount_pict[paramount_pict.profit == paramount_pict.profit.min()].original_title.iloc[0] + \
                " (" + paramount_pict[paramount_pict.profit == paramount_pict.profit.min()].imdb_id.iloc[0] + ")"
my_answer_18

'K-19: The Widowmaker (tt0267626)'

In [98]:
answer_to(answers, str(task_number), option_search(answer_options_18, my_answer_18))

answers[str(task_number)]

'1. K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [99]:
task_number = 19
answer_options_19 = [['2014'],
                    ['2008'],
                    ['2012'],
                    ['2002'],
                    ['2015']]

In [100]:
data.groupby('release_year').revenue.sum()[(data.groupby('release_year').revenue.sum()) == 
                                           (data.groupby('release_year').revenue.sum().max())]

release_year
2015    25449202382
Name: revenue, dtype: int64

In [101]:
my_answer_19 = data.groupby('release_year').revenue.sum()[(data.groupby('release_year').revenue.sum()) == 
                                           (data.groupby('release_year').revenue.sum().max())].index[0]
my_answer_19

2015

In [102]:
answer_to(answers, str(task_number), option_search(answer_options_19, str(my_answer_19)))

answers[str(task_number)]

'5. 2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [103]:
task_number = 20
answer_options_20 = [['2014'],
                    ['2008'],
                    ['2012'],
                    ['2010'],
                    ['2015']]

In [104]:
warner_bros = data[data.production_companies.str.contains("Warner Bros")]
warner_bros.groupby('release_year').profit.sum()[warner_bros.groupby('release_year').profit.sum() == 
                                                 warner_bros.groupby('release_year').profit.sum().max()]

release_year
2014    2295464519
Name: profit, dtype: int64

In [105]:
my_answer_20 = warner_bros.groupby('release_year').profit.sum()[warner_bros.groupby('release_year').profit.sum() == 
                                                 warner_bros.groupby('release_year').profit.sum().max()].index[0]
my_answer_20

2014

In [106]:
answer_to(answers, str(task_number), option_search(answer_options_20, str(my_answer_20)))

answers[str(task_number)]

'1. 2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [107]:
task_number = 21
answer_options_21 = [["Январь"],
                    ["Июнь"],
                    ["Декабрь"],
                    ["Сентябрь"],
                    ["Май"]]

In [108]:
data.release_date.str.split('/', n=1).str[0].value_counts(sort=True).head(1)

9    227
Name: release_date, dtype: int64

In [109]:
my_answer_21 = data.release_date.str.split('/', n=1).str[0].value_counts(sort=True).head(1).index[0]
my_answer_21

'9'

In [110]:
answer_to(answers, str(task_number), option_search(answer_options_21, month_convert(my_answer_21)))

answers[str(task_number)]

'4. Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [111]:
task_number = 22
answer_options_22 = [['345'],
                    ['450'],
                    ['478'],
                    ['523'],
                    ['381']]

In [112]:
my_answer_22 = data.release_date.str.split('/', n=1).str[0].value_counts().loc['6'] + \
    data.release_date.str.split('/', n=1).str[0].value_counts().loc['7'] + \
    data.release_date.str.split('/', n=1).str[0].value_counts().loc['8']
my_answer_22

450

In [113]:
answer_to(answers, str(task_number), option_search(answer_options_22, str(my_answer_22)))

answers[str(task_number)]

'2. 450'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [114]:
task_number = 23
answer_options_23 = [["Steven Soderbergh"],
                    ["Christopher Nolan"],
                    ["Clint Eastwood"],
                    ["Ridley Scott"],
                    ["Peter Jackson"]]

In [115]:
pd.Series(data.director[(data.release_date.str.split('/', n=1).str[0] == '1') | 
     (data.release_date.str.split('/', n=1).str[0] == '2') | 
     (data.release_date.str.split('/', n=1).str[0] == '12') ].str.split('|').sum()).value_counts().head(1)

Peter Jackson    7
dtype: int64

In [116]:
my_answer_23 = pd.Series(data.director[(data.release_date.str.split('/', n=1).str[0] == '1') | 
     (data.release_date.str.split('/', n=1).str[0] == '2') | 
     (data.release_date.str.split('/', n=1).str[0] == '12') ].str.split('|').sum()).value_counts().head(1).index[0]
my_answer_23

'Peter Jackson'

In [117]:
answer_to(answers, str(task_number), option_search(answer_options_23, my_answer_23))

answers[str(task_number)]

'5. Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [118]:
task_number = 24
answer_options_24 = [["Universal Pictures (Universal)"],
                    ["Warner Bros."],
                    ["Jim Henson Company, The"],
                    ["Paramount Pictures"],
                    ["Four By Two Productions"]]

In [119]:
df_company_title = pd.DataFrame(data[['original_title', 'production_companies']], 
                                columns = ['original_title', 'production_companies'])
df_company_title['length'] = df_company_title['original_title'].apply(lambda x: len(x))
df_company_title[df_company_title.length == df_company_title.length.max()]


,original_title,production_companies,length
1448,Borat: Cultural Learnings of America for Make ...,Twentieth Century Fox Film Corporation|Four By...,83


In [120]:
df_company_title[df_company_title.length == df_company_title.length.max()].production_companies.iloc[0].split('|')

['Twentieth Century Fox Film Corporation', 'Four By Two Productions']

In [121]:
my_answer_24 = df_company_title[df_company_title.length == df_company_title.length.max()].production_companies.\
                iloc[0].split('|')
my_answer_24

['Twentieth Century Fox Film Corporation', 'Four By Two Productions']

In [122]:
answer_to(answers, str(task_number), option_search(answer_options_24, my_answer_24))

answers[str(task_number)]

'5. Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [123]:
task_number = 25
answer_options_25 = [["Universal Pictures"],
                    ["Warner Bros"],
                    ["Midnight Picture Show"],
                    ["Paramount Pictures"],
                    ["Total Entertainment"]]

In [124]:
df_company_overview = pd.DataFrame(data[['overview', 'production_companies']], 
                                columns = ['overview', 'production_companies'])


In [125]:
df_company_overview['words'] = df_company_overview['overview'].apply(lambda x: len(x.split(' ')))
df_company_overview.head()

,overview,production_companies,words
0,Twenty-two years after the events of Jurassic ...,Universal Studios|Amblin Entertainment|Legenda...,26
1,An apocalyptic story set in the furthest reach...,Village Roadshow Pictures|Kennedy Miller Produ...,110
2,Beatrice Prior must confront her inner demons ...,Summit Entertainment|Mandeville Films|Red Wago...,22
3,Thirty years after defeating the Galactic Empi...,Lucasfilm|Truenorth Productions|Bad Robot,26
4,Deckard Shaw seeks revenge against Dominic Tor...,Universal Pictures|Original Film|Media Rights ...,14


In [126]:
df_company_overview_separated = pd.DataFrame({'production_companies':[], 'words':[]})
df_company_overview_separated

,production_companies,words


In [127]:
k = 0
for i, j in df_company_overview.iterrows():
    for company in j[1].split('|'):
        df_company_overview_separated.loc[k] = [company, j[2]]
        k += 1 

In [128]:
df_company_overview_separated.head(10)

,production_companies,words
0,Universal Studios,26.0
1,Amblin Entertainment,26.0
2,Legendary Pictures,26.0
3,Fuji Television Network,26.0
4,Dentsu,26.0
5,Village Roadshow Pictures,110.0
6,Kennedy Miller Productions,110.0
7,Summit Entertainment,22.0
8,Mandeville Films,22.0
9,Red Wagon Entertainment,22.0


In [129]:
df_company_overview_separated.groupby('production_companies').words.mean().sort_values().tail(1)

production_companies
Midnight Picture Show    175.0
Name: words, dtype: float64

In [130]:
my_answer_25 = df_company_overview_separated.groupby('production_companies').words.mean().sort_values().tail(1).index[0]
my_answer_25

'Midnight Picture Show'

In [131]:
answer_to(answers, str(task_number), option_search(answer_options_25, my_answer_25))

answers[str(task_number)]

'3. Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [132]:
task_number = 26
answer_options_26 = [['Inside Out', 'The Dark Knight', '12 Years a Slave'],
                ['BloodRayne', 'The Adventures of Rocky & Bullwinkle'],
                ['Batman Begins', 'The Lord of the Rings: The Return of the King', 'Upside Down'],
                ['300', 'Lucky Number Slevin', 'Kill Bill: Vol. 1'],
                ['Upside Down', 'Inside Out', 'Iron Man']]
                

In [133]:
indexes = list(data.vote_average.sort_values().tail(round(len(data) / 100)).index)
data.query("index in @indexes")[['original_title', 'vote_average']]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
155,The Theory of Everything,7.8
283,Mr. Nobody,7.8
370,Inception,7.9


In [134]:
my_answer_26 = data.query("index in @indexes").original_title.tolist()
my_answer_26

['Inside Out',
 'Room',
 'Interstellar',
 'Guardians of the Galaxy',
 'The Imitation Game',
 'Gone Girl',
 'The Grand Budapest Hotel',
 'The Theory of Everything',
 'Mr. Nobody',
 'Inception',
 'The Dark Knight',
 'The Lord of the Rings: The Two Towers',
 'The Pianist',
 'The Lord of the Rings: The Return of the King',
 'The Wolf of Wall Street',
 '12 Years a Slave',
 'Eternal Sunshine of the Spotless Mind',
 'There Will Be Blood',
 'Memento']

In [135]:
answer_to(answers, str(task_number), option_search(answer_options_26, my_answer_26))
answers[str(task_number)]

'1. Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [136]:
task_number = 27
answer_options_27 = [["Johnny Depp & Helena Bonham Carter"],
                ["Ben Stiller & Owen Wilson"],
                ["Vin Diesel & Paul Walker"],
                ["Adam Sandler & Kevin James"],
                ["Daniel Radcliffe & Rupert Grint"]]

In [137]:
data.cast.head(5)

0    Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...
1    Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...
2    Shailene Woodley|Theo James|Kate Winslet|Ansel...
3    Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...
4    Vin Diesel|Paul Walker|Jason Statham|Michelle ...
Name: cast, dtype: object

In [138]:
# Получим список всех пар актеров.
pair_actors = []
cast_list = data.cast.to_list()
for actors in cast_list:
    pair_up(actors.split('|'), pair_actors)
pair_actors[1:10]

['Bryce Dallas Howard & Irrfan Khan',
 'Bryce Dallas Howard & Nick Robinson',
 "Bryce Dallas Howard & Vincent D'Onofrio",
 'Chris Pratt & Irrfan Khan',
 'Chris Pratt & Nick Robinson',
 "Chris Pratt & Vincent D'Onofrio",
 'Irrfan Khan & Nick Robinson',
 "Irrfan Khan & Vincent D'Onofrio",
 "Nick Robinson & Vincent D'Onofrio"]

ВАРИАНТ 1

In [139]:
# Найдем кол-во повторений пар в списке. Результат словарь: ключ - пара, значение - кол-во повторов. 
pair_count = Counter(pair_actors)
pair_count

Counter({'Bryce Dallas Howard & Chris Pratt': 1,
         'Bryce Dallas Howard & Irrfan Khan': 1,
         'Bryce Dallas Howard & Nick Robinson': 1,
         "Bryce Dallas Howard & Vincent D'Onofrio": 1,
         'Chris Pratt & Irrfan Khan': 1,
         'Chris Pratt & Nick Robinson': 1,
         "Chris Pratt & Vincent D'Onofrio": 1,
         'Irrfan Khan & Nick Robinson': 1,
         "Irrfan Khan & Vincent D'Onofrio": 1,
         "Nick Robinson & Vincent D'Onofrio": 1,
         'Charlize Theron & Hugh Keays-Byrne': 1,
         'Charlize Theron & Josh Helman': 1,
         'Charlize Theron & Nicholas Hoult': 1,
         'Charlize Theron & Tom Hardy': 1,
         'Hugh Keays-Byrne & Josh Helman': 1,
         'Hugh Keays-Byrne & Nicholas Hoult': 1,
         'Hugh Keays-Byrne & Tom Hardy': 1,
         'Josh Helman & Nicholas Hoult': 1,
         'Josh Helman & Tom Hardy': 1,
         'Nicholas Hoult & Tom Hardy': 1,
         'Ansel Elgort & Kate Winslet': 1,
         'Ansel Elgort & Miles Te

In [140]:
# Найдем максимальное кол-во появления пар.
max_val = max(pair_count.values())
max_val

8

In [141]:
max_pair = {k:v for k, v in pair_count.items() if v == max_val}
max_pair


{'Daniel Radcliffe & Emma Watson': 8,
 'Daniel Radcliffe & Rupert Grint': 8,
 'Emma Watson & Rupert Grint': 8}

In [142]:
my_answer_27 = [x for x in max_pair.keys()]
my_answer_27

['Daniel Radcliffe & Emma Watson',
 'Daniel Radcliffe & Rupert Grint',
 'Emma Watson & Rupert Grint']

In [143]:
answer_to(answers, str(task_number), option_search(answer_options_27, my_answer_27))
answers[str(task_number)]

'5. Daniel Radcliffe & Rupert Grint'

ВАРИАНТ 2

In [144]:
# Альтернативный вариант решения реализуем через Series и value_counts().
pair_actors_series = pd.Series(pair_actors)
pair_actors_series.value_counts().max()

8

In [145]:
s = (pair_actors_series.value_counts() == pair_actors_series.value_counts().max())
s.loc[s].index.to_list()

['Daniel Radcliffe & Emma Watson',
 'Daniel Radcliffe & Rupert Grint',
 'Emma Watson & Rupert Grint']

# Submission

In [146]:
# Ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [147]:
# и убедиться что ни чего не пропустил)
len(answers)

27